# Script para que el usuario ponga su documento, genere las firmas o pueda validar.

### Obtener el path del archivo que elija el usuario

In [17]:
import  tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()
print(file_path)

D:/Documentos/pdf-test.pdf


### Función que obtiene el hash del documento

In [9]:
def hashPDF(file, BLOCK_SIZE):
    # hash=sha256()
    with open(file, 'rb') as f: # Open the file to read it's bytes
        fb = f.read(BLOCK_SIZE) # Read from the file. Take in the amount declared above
        M = sl.sha256(fb)
    return M

size = os.path.getsize(file_path) 

### Generación firma Schnorr

In [30]:
import create_keypair as ckp
import schnorr_lib as sl

user = ckp.create_keypair(1)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file_path, size)

sig = sl.schnorr_sign(M, user[0]["privateKey"])

print("PublicKey =",user[0]["publicKey"])
print("Signature =", sig.hex())

tiempos_firma = []
for i in range(1,100):
    t0 = time.perf_counter()
    M = hashPDF(file_path, size)
    t1 = time.perf_counter()
    total_2 = t1-t0
    tiempos_firma.append(total_2)

print(tiempos_firma)


PublicKey = 99e745bf433c9a86663c263d0c13a66f4b8f68470a5d48c84c04993e3c4ea112
Signature = 056c9b840c62a176fe0ce0f3d06654f0b4cc54ec74f8f36ac1b10be86a483dafdac565ae48692bb6bc4d6bc9f5d23f2c9d84ab21298aa36e32115b9c1ec8e21a
[0.0007944999961182475, 0.0006170999840833247, 0.0006546000076923519, 0.0006167000101413578, 0.0005612999957520515, 0.00042340002255514264, 0.0004335999838076532, 0.00045039999531582, 0.0004927999980282038, 0.00031430000672116876, 0.0005414000188466161, 0.0005265999934636056, 0.0004993999900761992, 0.0004200999974273145, 0.00042100000428035855, 0.0003836000105366111, 0.00038320000749081373, 0.0003809999907389283, 0.00038300000596791506, 0.00038240000139921904, 0.0003853000234812498, 0.0003816999960690737, 0.00038219999987632036, 0.0003803000145126134, 0.00038170002517290413, 0.000381199992261827, 0.00038109999150037766, 0.000381199992261827, 0.00042380002560094, 0.00037960000918246806, 0.0003806000167969614, 0.0003786000015679747, 0.00044589999015443027, 0.000405800004955

### Esquema Musig

In [34]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file_path, size)

sig, X, priv= sl.schnorr_musig_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

ValueError: not enough values to unpack (expected 3, got 2)

### Esquema Musig 2

In [15]:
import create_keypair as ckp
import schnorr_lib as sl
import os
# from hashlib import sha256

n_keys = int(input("Insert the number of keys to be generated:"))
# n_keys = 2
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file_path, size)

sig, X = sl.schnorr_musig2_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Aggregated key = 0c3db4a9cc99398f8b4d4e502fc63b718dc1d3cc3e6c6729b363b9433f1e5e34
Signature = 93a00a37929054819cf485662290b8e895c14ad24abb3a0ccc74d5a216c1548174b97900c22da530607f33b395d55e915edc65b6a87dd1d26d4f8d664bd3fd37


### Verificar Firma Schnorr

In [16]:
import schnorr_lib as sl

# M = input("Insert the message to verify:")
# M = sl.sha256(M.encode())
M = hashPDF(file_path, size)

pubkey = input("Insert the public key (or the aggregated key if MuSig was used):")
pubkey_bytes = bytes.fromhex(pubkey)

sig = input("Insert the generated sign:")
sig_bytes = bytes.fromhex(sig)

result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)

if result:
    print("The signature is VALID for this message and this public key")
else:
    print("The signature is NOT VALID for this message and this public key")

The signature is VALID for this message and this public key
